In [99]:
from google.colab import auth
auth.authenticate_user()

project_id = 'watchful-net-416319'
!gcloud config set project {project_id}

Updated property [core/project].


In [100]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import json
import requests
import logging
import threading


from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import AgglomerativeClustering


In [ ]:
!pip install matplotlib seaborn
!pip install google-cloud-storage
!pip install google-cloud-pubsub


In [ ]:
global df_cpu, df_gpu, df_ram, df_ssd, df_hdd

In [ ]:
from google.cloud import storage

# Initialize the GCS client
storage_client = storage.Client(project=project_id)

# Define your GCS bucket name
bucket_name = 'pc-component-files'

# Get the bucket
bucket = storage_client.get_bucket(bucket_name)

# List files in your bucket if needed
blobs = bucket.list_blobs()
print("Files in bucket:")
for blob in blobs:
    print(blob.name)

# Function to download a file from GCS to Colab environment
def download_from_gcs(source_blob_name, destination_file_name):
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Downloaded storage object {source_blob_name} from bucket {bucket_name} to local file {destination_file_name}.")

# Example: Download a specific file
download_from_gcs('edit google doc csv files/cleaned_CPU_UB.csv', '/content/cleaned_CPU_UB.csv')
download_from_gcs('edit google doc csv files/cleaned_GPU_UB.csv', '/content/cleaned_GPU_UB.csv')
download_from_gcs('edit google doc csv files/cleaned_SSD_UB.csv', '/content/cleaned_SSD_UB.csv')
download_from_gcs('edit google doc csv files/cleaned_HDD_UB.csv', '/content/cleaned_HDD_UB.csv')
download_from_gcs('edit google doc csv files/cleaned_RAM_UB.csv', '/content/cleaned_RAM_UB.csv')

download_from_gcs('clustered dataset/cleaned_CPU_UB_clustered.csv', '/content/cleaned_CPU_UB_clustered.csv')
download_from_gcs('clustered dataset/cleaned_GPU_UB_clustered.csv', '/content/cleaned_GPU_UB_clustered.csv')
download_from_gcs('clustered dataset/cleaned_HDD_UB_clustered.csv', '/content/cleaned_HDD_UB_clustered.csv')
download_from_gcs('clustered dataset/cleaned_RAM_UB_clustered.csv', '/content/cleaned_RAM_UB_clustered.csv')
download_from_gcs('clustered dataset/cleaned_SSD_UB_clustered.csv', '/content/cleaned_SSD_UB_clustered.csv')

In [ ]:
# Read each CSV file into a DataFrame
df_cpu = pd.read_csv('/content/cleaned_CPU_UB.csv')
df_gpu = pd.read_csv('/content/cleaned_GPU_UB.csv')
df_ssd = pd.read_csv('/content/cleaned_SSD_UB.csv')
df_hdd = pd.read_csv('/content/cleaned_HDD_UB.csv')
df_ram = pd.read_csv('/content/cleaned_RAM_UB.csv')

print("CPU Data:")
print(df_cpu.head())
print("\nGPU Data:")
print(df_gpu.head())
print("\nSSD Data:")
print(df_ssd.head())
print("\nHDD Data:")
print(df_hdd.head())
print("\nRAM Data:")
print(df_ram.head())


In [ ]:
cleaned_CPU_UB = pd.read_csv('/content/cleaned_CPU_UB_clustered.csv')
cleaned_GPU_UB = pd.read_csv('/content/cleaned_GPU_UB_clustered.csv')
cleaned_SSD_UB = pd.read_csv('/content/cleaned_HDD_UB_clustered.csv')
cleaned_HDD_UB = pd.read_csv('/content/cleaned_RAM_UB_clustered.csv')
cleaned_RAM_UB = pd.read_csv('/content/cleaned_SSD_UB_clustered.csv')

print("CPU Data:")
print(cleaned_CPU_UB.head())
print("\nGPU Data:")
print(cleaned_GPU_UB.head())
print("\nSSD Data:")
print(cleaned_SSD_UB.head())
print("\nHDD Data:")
print(cleaned_HDD_UB.head())
print("\nRAM Data:")
print(cleaned_RAM_UB.head())

In [ ]:
def calculate_correlation(file_path):
    # Read the dataset from the given file path
    data = pd.read_csv(file_path)

    # Select only numeric columns for correlation calculation
    numeric_data = data.select_dtypes(include=[np.number])

    # Calculate the correlation matrix
    correlation_matrix = numeric_data.corr()

    return correlation_matrix

# List of downloaded file paths
file_paths = [
    '/content/cleaned_CPU_UB_clustered.csv',
    '/content/cleaned_GPU_UB_clustered.csv',
    '/content/cleaned_HDD_UB_clustered.csv',
    '/content/cleaned_RAM_UB_clustered.csv',
    '/content/cleaned_SSD_UB_clustered.csv'
]

# Iterate through each file path
for file_path in file_paths:
    print(f"Correlation matrix for {file_path}:")
    correlation_matrix = calculate_correlation(file_path)
    print(correlation_matrix)
    print("\n")


In [ ]:
CPU_benchmark_data = cleaned_CPU_UB[['Benchmark']]
GPU_benchmark_data = cleaned_GPU_UB[['Benchmark']]
HDD_benchmark_data = cleaned_HDD_UB[['Benchmark']]
RAM_benchmark_data = cleaned_RAM_UB[['Benchmark']]
SSD_benchmark_data = cleaned_SSD_UB[['Benchmark']]

In [ ]:
def load_data(file_path):
    return pd.read_csv(file_path)

In [ ]:
# Standardizing the data using StandardScaler
scaler = StandardScaler()
CPU_benchmark_data_scaled = scaler.fit_transform(CPU_benchmark_data)
GPU_benchmark_data_scaled = scaler.fit_transform(GPU_benchmark_data)
SSD_benchmark_data_scaled = scaler.fit_transform(SSD_benchmark_data)
HDD_benchmark_data_scaled = scaler.fit_transform(HDD_benchmark_data)
RAM_benchmark_data_scaled = scaler.fit_transform(RAM_benchmark_data)

In [ ]:
# Based on the Elbow plot, choose an appropriate value for K and perform K-means clustering
k = 4  # Adjust this based on your Elbow plot observation
kmeans = KMeans(n_clusters=k, random_state=42)
cleaned_CPU_UB['Cluster'] = kmeans.fit_predict(CPU_benchmark_data_scaled)

In [ ]:
# Based on the Elbow plot, choose an appropriate value for K and perform K-means clustering
k = 4  # Adjust this based on your Elbow plot observation
kmeans = KMeans(n_clusters=k, random_state=42)
cleaned_GPU_UB['Cluster'] = kmeans.fit_predict(GPU_benchmark_data_scaled)

In [ ]:
# Based on the Elbow plot, choose an appropriate value for K and perform K-means clustering
k = 4  # Adjust this based on your Elbow plot observation
kmeans = KMeans(n_clusters=k, random_state=42)
cleaned_HDD_UB['Cluster'] = kmeans.fit_predict(HDD_benchmark_data_scaled)

In [ ]:
# Based on the Elbow plot, choose an appropriate value for K and perform K-means clustering
k = 4  # Adjust this based on your Elbow plot observation
kmeans = KMeans(n_clusters=k, random_state=42)
cleaned_SSD_UB['Cluster'] = kmeans.fit_predict(SSD_benchmark_data_scaled)

In [ ]:
# Based on the Elbow plot, choose an appropriate value for K and perform K-means clustering
k = 4  # Adjust this based on your Elbow plot observation
kmeans = KMeans(n_clusters=k, random_state=42)
cleaned_RAM_UB['Cluster'] = kmeans.fit_predict(RAM_benchmark_data_scaled)

In [ ]:

# Splitting CPU benchmark data
CPU_train, CPU_test = train_test_split(CPU_benchmark_data_scaled, test_size=0.2, random_state=42)

# Splitting GPU benchmark data
GPU_train, GPU_test = train_test_split(GPU_benchmark_data_scaled, test_size=0.2, random_state=42)

# Splitting SSD benchmark data
SSD_train, SSD_test = train_test_split(SSD_benchmark_data_scaled, test_size=0.2, random_state=42)

# Splitting HDD benchmark data
HDD_train, HDD_test = train_test_split(HDD_benchmark_data_scaled, test_size=0.2, random_state=42)

# Splitting RAM benchmark data
RAM_train, RAM_test = train_test_split(RAM_benchmark_data_scaled, test_size=0.2, random_state=42)


In [ ]:
# Function to apply clustering and integrate labels
def apply_clustering_and_integrate(X_train):
    # Apply DBSCAN clustering
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    clusters_dbscan = dbscan.fit_predict(X_train)

    # Apply Hierarchical clustering
    hierarchical = AgglomerativeClustering(n_clusters=4)  # Adjust based on analysis
    clusters_hierarchical = hierarchical.fit_predict(X_train)

    # Integrate DBSCAN cluster labels into the dataset
    X_train_with_dbscan = np.column_stack((X_train, clusters_dbscan))

    # Integrate Hierarchical cluster labels into the dataset
    X_train_with_hierarchical = np.column_stack((X_train, clusters_hierarchical))

    return X_train_with_dbscan, X_train_with_hierarchical


In [ ]:
# Applying clustering to each training dataset and integrating labels
CPU_train_with_dbscan, CPU_train_with_hierarchical = apply_clustering_and_integrate(CPU_train)
GPU_train_with_dbscan, GPU_train_with_hierarchical = apply_clustering_and_integrate(GPU_train)
SSD_train_with_dbscan, SSD_train_with_hierarchical = apply_clustering_and_integrate(SSD_train)
HDD_train_with_dbscan, HDD_train_with_hierarchical = apply_clustering_and_integrate(HDD_train)
RAM_train_with_dbscan, RAM_train_with_hierarchical = apply_clustering_and_integrate(RAM_train)

In [ ]:
# Creating DataFrames for each dataset with hierarchical cluster labels
CPU_train_df_hier = pd.DataFrame(CPU_train_with_hierarchical, columns=['Scaled_Benchmark', 'Hierarchical_Cluster'])
GPU_train_df_hier = pd.DataFrame(GPU_train_with_hierarchical, columns=['Scaled_Benchmark', 'Hierarchical_Cluster'])
SSD_train_df_hier = pd.DataFrame(SSD_train_with_hierarchical, columns=['Scaled_Benchmark', 'Hierarchical_Cluster'])
HDD_train_df_hier = pd.DataFrame(HDD_train_with_hierarchical, columns=['Scaled_Benchmark', 'Hierarchical_Cluster'])
RAM_train_df_hier = pd.DataFrame(RAM_train_with_hierarchical, columns=['Scaled_Benchmark', 'Hierarchical_Cluster'])

# Display the first few rows for each DataFrame
print("CPU Data (Hierarchical Clustering):")
print(CPU_train_df_hier.head())
print("\nGPU Data (Hierarchical Clustering):")
print(GPU_train_df_hier.head())
print("\nSSD Data (Hierarchical Clustering):")
print(SSD_train_df_hier.head())
print("\nHDD Data (Hierarchical Clustering):")
print(HDD_train_df_hier.head())
print("\nRAM Data (Hierarchical Clustering):")
print(RAM_train_df_hier.head())


In [ ]:
def plot_clusters(X, labels, title='Clustering Visualization'):
    # Check if the data is 1D or 2D+
    if X.ndim > 1 and X.shape[1] > 1:
        # If data is multidimensional, apply PCA to reduce to two dimensions
        pca = PCA(n_components=2)
        X_transformed = pca.fit_transform(X)
        x_axis = X_transformed[:, 0]
        y_axis = X_transformed[:, 1]
    else:
        # If data is 1D, plot directly or duplicate the 1D data for x and use zeros for y
        if X.ndim > 1:
            x_axis = X[:, 0]
        else:
            x_axis = X  # If X is a 1D array already
        y_axis = np.zeros(len(x_axis))  # Use zeros for y-axis

    plt.figure(figsize=(10, 8))
    sns.scatterplot(x=x_axis, y=y_axis, hue=labels, palette='viridis', legend='full', s=100)
    plt.title(title)
    plt.xlabel('Scaled Benchmark' if X.ndim == 1 or X.shape[1] == 1 else 'Dimension 1')
    plt.ylabel('Dimension 2' if X.ndim > 1 and X.shape[1] > 1 else '')
    plt.yticks([])  # Hide y-axis ticks for 1D data
    plt.legend(title='Cluster')
    plt.show()


In [ ]:
def calculate_and_print_silhouette_score(X, labels, clustering_type, data_label):
    # Filter out noise for DBSCAN (-1 labels)
    if clustering_type == 'DBSCAN':
        filtered_labels = labels[labels != -1]
        if len(np.unique(filtered_labels)) < 2:
            print(f"Silhouette Score for {data_label} with {clustering_type}: Cannot be computed (less than 2 clusters).")
            return
        filtered_X = X[labels != -1, :]
    else:
        filtered_labels = labels
        filtered_X = X

    # Calculate Silhouette Score
    score = silhouette_score(filtered_X, filtered_labels)
    print(f"Silhouette Score for {data_label} with {clustering_type}: {score:.3f}")

In [ ]:
# Assuming *_train are your datasets and *_train_df['DBSCAN_Cluster'], *_train_df_hier['Hierarchical_Cluster']
# contain the respective clustering labels
# For Hierarchical clustering results
calculate_and_print_silhouette_score(CPU_train, CPU_train_df_hier['Hierarchical_Cluster'], 'Hierarchical', 'CPU')
calculate_and_print_silhouette_score(GPU_train, GPU_train_df_hier['Hierarchical_Cluster'], 'Hierarchical', 'GPU')
calculate_and_print_silhouette_score(HDD_train, HDD_train_df_hier['Hierarchical_Cluster'], 'Hierarchical', 'HDD')
calculate_and_print_silhouette_score(SSD_train, SSD_train_df_hier['Hierarchical_Cluster'], 'Hierarchical', 'SSD')
calculate_and_print_silhouette_score(RAM_train, RAM_train_df_hier['Hierarchical_Cluster'], 'Hierarchical', 'RAM')

# Continue for SSD, HDD, RAM


In [ ]:
# Print all column names in the DataFrame to verify the correct column names
print(df_cpu.columns)


CPU RECOMMENDATION SYSTEM


In [ ]:
numeric_features = ['Benchmark']  # Assuming these are correct
categorical_features = ['Model_Type','Tier', 'Processor_Number']  # Correct based on actual column names in DataFrame

for feature in numeric_features:
    if df_cpu[feature].dtype == 'object':
        df_cpu[feature] = pd.to_numeric(df_cpu[feature], errors='coerce')
        # The print statements should be indented to be part of the if block
        print(f"Non-numeric values found in {feature}:")
        print(df_cpu[feature].unique())  # This will show all unique values in the column

# Define your transformations
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer
preprocessor_cpu = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Now you can proceed to transform your data
X_processed_cpu = preprocessor_cpu.fit_transform(df_cpu)

# If you need the transformed DataFrame (for example, to check or use the transformed features)
columns_transformed = (numeric_features +
                       list(preprocessor_cpu.named_transformers_['cat'].get_feature_names_out(categorical_features)))
X_processed_df_cpu = pd.DataFrame(X_processed_cpu.toarray(), columns=columns_transformed)
print(X_processed_df_cpu)

# Assuming X_processed_df is your processed DataFrame
output_file_path = 'processed_cpu_data.csv'  # Specify your file path here
X_processed_df_cpu.to_csv(output_file_path, index=False)


In [ ]:
# Convert X_dense to a numpy array if it isn't already one
X_dense_cpu = X_processed_cpu.toarray() if hasattr(X_processed_cpu, "toarray") else np.array(X_processed_cpu)


# Replace NaN values with the mean of each column, handling columns with all NaNs separately
for i in range(X_dense_cpu.shape[1]):
    column = X_dense_cpu[:, i]
    nan_mask = np.isnan(column)
    if nan_mask.all():
        X_dense_cpu[:, i] = 0  # Replace all NaN columns with 0 or another appropriate value
    else:
        column_mean = np.nanmean(column)
        column[nan_mask] = column_mean  # Replace NaNs in the column with the mean of the non-NaN values


# Ensure no infinite values exist, replacing them if they do
inf_mask = np.isinf(X_dense_cpu)
X_dense_cpu[inf_mask] = np.nan  # Convert infinities to NaN
np.nan_to_num(X_dense_cpu, copy=False, nan=0.0)  # Then replace NaNs with 0 (or another value)


# Now proceed with hierarchical clustering
linked = linkage(X_dense_cpu, 'ward')
labelList = range(1, 11)

# Generate labels dynamically based on the number of rows in X_dense
labelList = [f"Point {i+1}" for i in range(X_dense_cpu.shape[0])]



In [ ]:
# Assuming df_cpu is your DataFrame containing the data
# Define which are your numeric and categorical features correctly
numeric_features = ['Benchmark']  # This should be your actual numeric feature(s)
categorical_features = ['Model_Type', 'Tier', 'Processor_Number']  # Including 'Tier' and 'Processor_Number' as categorical

# Create pipelines for both numeric and categorical preprocessing
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer with appropriate transformations
preprocessor_cpu = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Apply preprocessing
X_processed_cpu = preprocessor_cpu.fit_transform(df_cpu)  # No need to call .toarray() here yet

# If you need a dense format, especially for hierarchical clustering, convert to array
X_processed_dense_cpu = X_processed_cpu.toarray()

# Hierarchical Clustering to create a linkage matrix 'Z'
Z = linkage(X_processed_dense_cpu, method='ward')


# Find the largest distance gap in the dendrogram to suggest an optimal distance
distances = Z[:, 2]
distance_diffs = np.diff(-np.sort(distances))
idx_of_largest_gap = np.argmax(distance_diffs)
optimal_distance = distances[idx_of_largest_gap]
clusters = fcluster(Z, optimal_distance, criterion='distance')

# Find the optimal number of clusters using silhouette scores
max_silhouette_score = -1
optimal_num_clusters = 81

##for num_clusters in range(2, len(np.unique(clusters)) + 1):
  ##  cluster_labels = fcluster(Z, num_clusters, criterion='maxclust')
    ##silhouette_avg = silhouette_score(X_processed, cluster_labels)
    ##print(f'For n_clusters = {num_clusters}, the average silhouette_score is : {silhouette_avg}')
    ##if silhouette_avg > max_silhouette_score:
      ##  max_silhouette_score = silhouette_avg
        #optimal_num_clusters = 81

print(f"The optimal number of clusters based on silhouette score is: {optimal_num_clusters}") #81 clusters

In [ ]:
# Apply Agglomerative Clustering
cluster_cpu= AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')
cluster_labels_cpu = cluster_cpu.fit_predict(X_processed_dense)  # Use X_processed_dense here

print(cluster_labels_cpu)

In [ ]:
print(f"Number of rows in df_cpu: {len(df_cpu)}")

In [ ]:
# Example: If you dropped NA values before clustering
df_cpu_cleaned = df_cpu.dropna(subset=['Benchmark', 'Model_Type', 'Tier', 'Processor_Number'])

# Assuming the generation of cluster_labels here aligns with your data
if len(cluster_labels_cpu) == len(df_cpu):
    df_cpu['cluster'] = cluster_labels_cpu
    print("Cluster labels successfully assigned.")
else:
    print(f"Mismatch detected: {len(cluster_labels_cpu)} cluster labels vs {len(df_cpu)} rows in DataFrame.")
    # You might need to revisit the clustering step or ensure preprocessing steps are aligned


In [ ]:
# Assuming df_cpu is your DataFrame
# First, convert columns that should be numeric but are 'object' type
#for col in df_cpu.columns:
 #   if df_cpu[col].dtype == 'object':
        # Attempt to convert to numeric, coerce errors will turn non-convertible values into NaN
  #      df_cpu[col] = pd.to_numeric(df_cpu[col], errors='coerce')

# Now exclude any remaining non-numeric columns before grouping
#numeric_cols = df_cpu.select_dtypes(include=[np.number])

# Calculating centroids of numeric columns only
#centroids = numeric_cols.groupby('cluster').mean()

#print(centroids)

In [ ]:
df_cpu['cluster_cpu'] = cluster_labels_cpu


GPU RECOMMENDATION SYSTEM

In [ ]:
print(df_gpu.columns)

In [ ]:
# Adjust the column names as per your GPU dataset
numeric_features_gpu = ['Benchmark']  # Assuming Benchmark is a numeric feature for GPUs as well
categorical_features_gpu = ['GPU_Model_Type', 'VRAM_Size', 'GPU_Model_Tier']  # Example categorical features for GPUs, adjust as needed

for feature in numeric_features_gpu:
    if df_gpu[feature].dtype == 'object':
        df_gpu[feature] = pd.to_numeric(df_gpu[feature], errors='coerce')
        print(f"Non-numeric values found in {feature}:")
        print(df_gpu[feature].unique())

# Define your transformations for GPU data
numeric_transformer_gpu = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer_gpu = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer for GPU data
preprocessor_gpu = ColumnTransformer(transformers=[
    ('num', numeric_transformer_gpu, numeric_features_gpu),
    ('cat', categorical_transformer_gpu, categorical_features_gpu)
])

# Now you can proceed to transform your GPU data
X_processed_gpu = preprocessor_gpu.fit_transform(df_gpu)

# If you need the transformed DataFrame (for example, to check or use the transformed features for GPUs)
columns_transformed_gpu = (numeric_features_gpu +
                           list(preprocessor_gpu.named_transformers_['cat'].get_feature_names_out(categorical_features_gpu)))
X_processed_gpu_df = pd.DataFrame(X_processed_gpu.toarray(), columns=columns_transformed_gpu)
print(X_processed_gpu_df)

# Assuming X_processed_gpu_df is your processed DataFrame for GPU data
output_file_path_gpu = 'processed_gpu_data.csv'  # Specify your file path here
X_processed_gpu_df.to_csv(output_file_path_gpu, index=False)

In [ ]:
# Convert X_processed_gpu to a numpy array if it isn't already one
X_dense_gpu = X_processed_gpu.toarray() if hasattr(X_processed_gpu, "toarray") else np.array(X_processed_gpu)

# Replace NaN values with the mean of each column, handling columns with all NaNs separately for GPU data
for i in range(X_dense_gpu.shape[1]):
    column = X_dense_gpu[:, i]
    nan_mask = np.isnan(column)
    if nan_mask.all():
        X_dense_gpu[:, i] = 0  # Replace all NaN columns with 0 or another appropriate value
    else:
        column_mean = np.nanmean(column)
        column[nan_mask] = column_mean  # Replace NaNs in the column with the mean of the non-NaN values

# Ensure no infinite values exist, replacing them if they do for GPU data
inf_mask = np.isinf(X_dense_gpu)
X_dense_gpu[inf_mask] = np.nan  # Convert infinities to NaN
np.nan_to_num(X_dense_gpu, copy=False, nan=0.0)  # Then replace NaNs with 0 (or another value)

# Now proceed with hierarchical clustering for GPU data
linked_gpu = linkage(X_dense_gpu, 'ward')
labelList_gpu = range(1, 11)

# Generate labels dynamically based on the number of rows in X_dense_gpu
labelList_gpu = [f"Point {i+1}" for i in range(X_dense_gpu.shape[0])]

In [ ]:
# Assuming df_gpu is your DataFrame containing the GPU data
# Adjust your numeric and categorical features accordingly
numeric_features_gpu = ['Benchmark']  # Benchmark remains a numeric feature
categorical_features_gpu = ['GPU_Model_Type', 'VRAM_Size', 'GPU_Model_Tier']  # Adjusted for GPU-specific features

# Create pipelines for both numeric and categorical preprocessing for GPU
numeric_transformer_gpu = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer_gpu = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer with appropriate transformations for GPU
preprocessor_gpu = ColumnTransformer(transformers=[
    ('num', numeric_transformer_gpu, numeric_features_gpu),
    ('cat', categorical_transformer_gpu, categorical_features_gpu)
])

# Apply preprocessing to the GPU data
X_processed_gpu = preprocessor_gpu.fit_transform(df_gpu)  # This gets you the transformed data

# Convert to dense format if necessary, especially for hierarchical clustering
X_processed_dense_gpu = X_processed_gpu.toarray()

# Hierarchical Clustering to create a linkage matrix 'Z' for GPU data
Z_gpu = linkage(X_processed_dense_gpu, method='ward')

# Plot the dendrogram for visual inspection for GPU data
#plt.figure(figsize=(10, 7))
#dendrogram(Z_gpu)
#plt.title('Hierarchical Clustering Dendrogram for GPU Data')
#plt.xlabel('Sample index')
#plt.ylabel('Distance')
#plt.show()

# Suggesting an optimal distance and finding the number of clusters for GPU data
distances_gpu = Z_gpu[:, 2]
distance_diffs_gpu = np.diff(-np.sort(distances_gpu))
idx_of_largest_gap_gpu = np.argmax(distance_diffs_gpu)
optimal_distance_gpu = distances_gpu[idx_of_largest_gap_gpu]
clusters_gpu = fcluster(Z_gpu, optimal_distance_gpu, criterion='distance')

# Determine the optimal number of clusters using silhouette scores for GPU data
max_silhouette_score_gpu = -1
optimal_num_clusters_gpu = 263

#for num_clusters in range(2, len(np.unique(clusters_gpu)) + 1):
#    cluster_labels_gpu = fcluster(Z_gpu, num_clusters, criterion='maxclust')
#    silhouette_avg_gpu = silhouette_score(X_processed_gpu, cluster_labels_gpu)
 #   print(f'For n_clusters = {num_clusters}, the average silhouette_score is : {silhouette_avg_gpu}')
  #  if silhouette_avg_gpu > max_silhouette_score_gpu:
   #     max_silhouette_score_gpu = silhouette_avg_gpu
    #    optimal_num_clusters_gpu = num_clusters

print(f"The optimal number of clusters based on silhouette score for GPU data is: {optimal_num_clusters_gpu}")#


In [ ]:
# Apply Agglomerative Clustering to GPU data
cluster_gpu = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')
cluster_labels_gpu = cluster_gpu.fit_predict(X_processed_dense_gpu)  # Use the processed dense GPU data here

print(cluster_labels_gpu)


In [ ]:
# Assuming df_cpu is the original DataFrame and cluster_labels contains the cluster assignments
df_gpu['cluster_gpu'] = cluster_labels_gpu

# Calculating centroids
#GPU_centroids = df_gpu.groupby('cluster_gpu').mean()

#print(GPU_centroids)

RAM RECOMMENDATION SYSTEM

In [ ]:
print(df_ram.columns)

In [ ]:
# Assuming df_ram is your RAM dataset DataFrame
# Load your RAM dataset here, for example:
# df_ram = pd.read_csv('path_to_your_ram_dataset.csv')

# Define numeric and categorical features for RAM
numeric_features_ram = ['Benchmark']
categorical_features_ram = ['Type_Speed', 'CAS_Latency', 'RAM_Sticks', 'GB_Amount']

# Preprocess numeric features to ensure they are numeric, especially if they are imported as strings
for feature in numeric_features_ram:
    if df_ram[feature].dtype == 'object':
        df_ram[feature] = pd.to_numeric(df_ram[feature], errors='coerce')
        print(f"Non-numeric values found in {feature}:")
        print(df_ram[feature].unique())

# Create preprocessing pipelines for RAM
numeric_transformer_ram = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer_ram = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer for RAM data
preprocessor_ram = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_ram, numeric_features_ram),
        ('cat', categorical_transformer_ram, categorical_features_ram)])

# Preprocess the RAM dataset
X_processed_ram = preprocessor_ram.fit_transform(df_ram)

# Convert the processed data into a dense DataFrame for easier handling and visualization
columns_transformed_ram = numeric_features_ram + \
    list(preprocessor_ram.named_transformers_['cat'].get_feature_names_out(categorical_features_ram))
X_processed_ram_df = pd.DataFrame(X_processed_ram.toarray(), columns=columns_transformed_ram)
print(X_processed_ram_df)

# Save the processed RAM DataFrame to a CSV file
output_file_path_ram = 'processed_ram_data.csv'  # Specify your file path here
X_processed_ram_df.to_csv(output_file_path_ram, index=False)


In [ ]:
# Convert X_processed_ram to a numpy array if it isn't already one
X_dense_ram = X_processed_ram.toarray() if hasattr(X_processed_ram, "toarray") else np.array(X_processed_ram)

# Replace NaN values with the mean of each column, handling columns with all NaNs separately for RAM data
for i in range(X_dense_ram.shape[1]):
    column = X_dense_ram[:, i]
    nan_mask = np.isnan(column)
    if nan_mask.all():
        X_dense_ram[:, i] = 0  # Replace all NaN columns with 0 or another appropriate value
    else:
        column_mean = np.nanmean(column)
        column[nan_mask] = column_mean  # Replace NaNs in the column with the mean of the non-NaN values

# Ensure no infinite values exist, replacing them if they do for RAM data
inf_mask = np.isinf(X_dense_ram)
X_dense_ram[inf_mask] = np.nan  # Convert infinities to NaN
np.nan_to_num(X_dense_ram, copy=False, nan=0.0)  # Then replace NaNs with 0 (or another value)

# Now proceed with hierarchical clustering for RAM data
linked_ram = linkage(X_dense_ram, 'ward')

# Assuming you want to dynamically generate labels for a plot or analysis
# Here we prepare labelList_ram for potentially labeling points in a plot
labelList_ram = [f"Point {i+1}" for i in range(X_dense_ram.shape[0])]

In [ ]:
# Assuming df_ram is your DataFrame containing the RAM data
# Define numeric and categorical features for RAM
numeric_features_ram = ['Benchmark']  # Benchmark is a numeric feature for RAM as well
categorical_features_ram = ['Type_Speed', 'CAS_Latency', 'RAM_Sticks', 'GB_Amount']  # Specific features for RAM

# Create pipelines for both numeric and categorical preprocessing for RAM
numeric_transformer_ram = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer_ram = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer with appropriate transformations for RAM
preprocessor_ram = ColumnTransformer(transformers=[
    ('num', numeric_transformer_ram, numeric_features_ram),
    ('cat', categorical_transformer_ram, categorical_features_ram)])

# Apply preprocessing to the RAM data
X_processed_ram = preprocessor_ram.fit_transform(df_ram)  # This gets you the transformed RAM data

# Convert to dense format if necessary, especially for hierarchical clustering
X_processed_dense_ram = X_processed_ram.toarray()

# Hierarchical Clustering to create a linkage matrix 'Z' for RAM data
Z_ram = linkage(X_processed_dense_ram, method='ward')

# Plot the dendrogram for visual inspection for RAM data
#plt.figure(figsize=(10, 7))
#dendrogram(Z_ram)
#plt.title('Hierarchical Clustering Dendrogram for RAM Data')
#plt.xlabel('Sample index')
#plt.ylabel('Distance')
#plt.show()

# Suggesting an optimal distance and finding the number of clusters for RAM data
distances_ram = Z_ram[:, 2]
distance_diffs_ram = np.diff(-np.sort(distances_ram))
idx_of_largest_gap_ram = np.argmax(distance_diffs_ram)
optimal_distance_ram = distances_ram[idx_of_largest_gap_ram]
clusters_ram = fcluster(Z_ram, optimal_distance_ram, criterion='distance')

# Determine the optimal number of clusters using silhouette scores for RAM data
max_silhouette_score_ram = -1
optimal_num_clusters_ram = 59
#for num_clusters_ram in range(2, len(np.unique(clusters_ram)) + 1):
 #   cluster_labels_ram = fcluster(Z_ram, num_clusters_ram, criterion='maxclust')
  #  silhouette_avg_ram = silhouette_score(X_processed_ram, cluster_labels_ram)
   # if silhouette_avg_ram > max_silhouette_score_ram:
    #    max_silhouette_score_ram = silhouette_avg_ram
     #   optimal_num_clusters_ram = num_clusters_ram

print(f"The optimal number of clusters based on silhouette score for RAM data is: {optimal_num_clusters_ram}")

In [ ]:
# Apply Agglomerative Clustering to RAM data
cluster_ram = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')
cluster_labels_ram = cluster_ram.fit_predict(X_processed_dense_ram)  # Use the processed dense RAM data here

print(cluster_labels_ram)


In [ ]:
# Assuming df_ram is the original DataFrame and cluster_labels_ram contains the cluster assignments
df_ram['cluster_ram'] = cluster_labels_ram

# Calculating centroids for RAM data
#RAM_centroids = df_ram.groupby('cluster_ram').mean()

#print(RAM_centroids)


SSD RECOMMENDATION SYSTEM


In [ ]:
print(df_ssd.columns)

In [ ]:
# Assuming df_ssd is your SSD dataset DataFrame
# Load your SSD dataset here, for example:
# df_ssd = pd.read_csv('path_to_your_ssd_dataset.csv')

# Define numeric and categorical features for SSD
numeric_features_ssd = ['Benchmark']
categorical_features_ssd = ['Size', 'Model_Type']

# Preprocess numeric features to ensure they are numeric, especially if they are imported as strings
for feature in numeric_features_ssd:
    if df_ssd[feature].dtype == 'object':
        df_ssd[feature] = pd.to_numeric(df_ssd[feature], errors='coerce')
        print(f"Non-numeric values found in {feature}:")
        print(df_ssd[feature].unique())

# Create preprocessing pipelines for SSD
numeric_transformer_ssd = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer_ssd = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer for SSD data
preprocessor_ssd = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_ssd, numeric_features_ssd),
        ('cat', categorical_transformer_ssd, categorical_features_ssd)])

# Preprocess the SSD dataset
X_processed_ssd = preprocessor_ssd.fit_transform(df_ssd)

# Convert the processed data into a dense DataFrame for easier handling and visualization
columns_transformed_ssd = numeric_features_ssd + \
    list(preprocessor_ssd.named_transformers_['cat'].get_feature_names_out(categorical_features_ssd))
X_processed_ssd_df = pd.DataFrame(X_processed_ssd.toarray(), columns=columns_transformed_ssd)
print(X_processed_ssd_df)

# Save the processed SSD DataFrame to a CSV file
output_file_path_ssd = 'processed_ssd_data.csv'  # Specify your file path here
X_processed_ssd_df.to_csv(output_file_path_ssd, index=False)

In [ ]:
# Convert X_processed_ssd to a numpy array if it isn't already one
X_dense_ssd = X_processed_ssd.toarray() if hasattr(X_processed_ssd, "toarray") else np.array(X_processed_ssd)

# Replace NaN values with the mean of each column, handling columns with all NaNs separately for SSD data
for i in range(X_dense_ssd.shape[1]):
    column = X_dense_ssd[:, i]
    nan_mask = np.isnan(column)
    if nan_mask.all():
        X_dense_ssd[:, i] = 0  # Replace all NaN columns with 0 or another appropriate value
    else:
        column_mean = np.nanmean(column)
        column[nan_mask] = column_mean  # Replace NaNs in the column with the mean of the non-NaN values

# Ensure no infinite values exist, replacing them if they do for SSD data
inf_mask = np.isinf(X_dense_ssd)
X_dense_ssd[inf_mask] = np.nan  # Convert infinities to NaN
np.nan_to_num(X_dense_ssd, copy=False, nan=0.0)  # Then replace NaNs with 0 (or another value)

# Now proceed with hierarchical clustering for SSD data
linked_ssd = linkage(X_dense_ssd, 'ward')

# Assuming you want to dynamically generate labels for a plot or analysis
# Here we prepare labelList_ssd for potentially labeling points in a plot
labelList_ssd = [f"Point {i+1}" for i in range(X_dense_ssd.shape[0])]

In [ ]:
# Assuming df_ssd is your DataFrame containing the SSD data
# Define numeric and categorical features for SSD
numeric_features_ssd = ['Benchmark']  # Benchmark is a numeric feature for SSD as well
categorical_features_ssd = ['Size', 'Model_Type']  # Specific features for SSD

# Create pipelines for both numeric and categorical preprocessing for SSD
numeric_transformer_ssd = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer_ssd = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer with appropriate transformations for SSD
preprocessor_ssd = ColumnTransformer(transformers=[
    ('num', numeric_transformer_ssd, numeric_features_ssd),
    ('cat', categorical_transformer_ssd, categorical_features_ssd)])

# Apply preprocessing to the SSD data
X_processed_ssd = preprocessor_ssd.fit_transform(df_ssd)  # This gets you the transformed SSD data

# Convert to dense format if necessary, especially for hierarchical clustering
X_processed_dense_ssd = X_processed_ssd.toarray()

# Hierarchical Clustering to create a linkage matrix 'Z' for SSD data
Z_ssd = linkage(X_processed_dense_ssd, method='ward')

# Plot the dendrogram for visual inspection for SSD data
#plt.figure(figsize=(10, 7))
#dendrogram(Z_ssd)
#plt.title('Hierarchical Clustering Dendrogram for SSD Data')
#plt.xlabel('Sample index')
#plt.ylabel('Distance')
#plt.show()

# Suggesting an optimal distance and finding the number of clusters for SSD data
distances_ssd = Z_ssd[:, 2]
distance_diffs_ssd = np.diff(-np.sort(distances_ssd))
idx_of_largest_gap_ssd = np.argmax(distance_diffs_ssd)
optimal_distance_ssd = distances_ssd[idx_of_largest_gap_ssd]
clusters_ssd = fcluster(Z_ssd, optimal_distance_ssd, criterion='distance')

# Determine the optimal number of clusters using silhouette scores for SSD data
max_silhouette_score_ssd = -1
optimal_num_clusters_ssd = 63
#for num_clusters_ssd in range(2, len(np.unique(clusters_ssd)) + 1):
 #   cluster_labels_ssd = fcluster(Z_ssd, num_clusters_ssd, criterion='maxclust')
  #  silhouette_avg_ssd = silhouette_score(X_processed_ssd, cluster_labels_ssd)
   # if silhouette_avg_ssd > max_silhouette_score_ssd:
    #    max_silhouette_score_ssd = silhouette_avg_ssd
     #   optimal_num_clusters_ssd = num_clusters_ssd

print(f"The optimal number of clusters based on silhouette score for SSD data is: {optimal_num_clusters_ssd}")


In [ ]:
# Apply Agglomerative Clustering to SSD data
cluster_ssd = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')
cluster_labels_ssd = cluster_ssd.fit_predict(X_processed_dense_ssd)  # Use the processed dense SSD data here

print(cluster_labels_ssd)


In [ ]:
# Assuming df_ssd is the original DataFrame and cluster_labels_ssd contains the cluster assignments
df_ssd['cluster_ssd'] = cluster_labels_ssd

# Calculating centroids for SSD data
#SSD_centroids = df_ssd.groupby('cluster_ssd').mean()

#print(SSD_centroids)



HDD RECOMMENDATION SYSTEM

In [ ]:
print(df_hdd.columns)

In [ ]:
# Assuming df_hdd is your HDD dataset DataFrame
# Load your HDD dataset here, for example:
# df_hdd = pd.read_csv('path_to_your_hdd_dataset.csv')

# Define numeric and categorical features for HDD
numeric_features_hdd = ['Benchmark']
categorical_features_hdd = ['Storage_Size', 'Model_Name']

# Preprocess numeric features to ensure they are numeric, especially if they are imported as strings
for feature in numeric_features_hdd:
    if df_hdd[feature].dtype == 'object':
        df_hdd[feature] = pd.to_numeric(df_hdd[feature], errors='coerce')
        print(f"Non-numeric values found in {feature}:")
        print(df_hdd[feature].unique())

# Create preprocessing pipelines for HDD
numeric_transformer_hdd = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer_hdd = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer for HDD data
preprocessor_hdd = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_hdd, numeric_features_hdd),
        ('cat', categorical_transformer_hdd, categorical_features_hdd)])

# Preprocess the HDD dataset
X_processed_hdd = preprocessor_hdd.fit_transform(df_hdd)

# Convert the processed data into a dense DataFrame for easier handling and visualization
columns_transformed_hdd = numeric_features_hdd + \
    list(preprocessor_hdd.named_transformers_['cat'].get_feature_names_out(categorical_features_hdd))
X_processed_hdd_df = pd.DataFrame(X_processed_hdd.toarray(), columns=columns_transformed_hdd)
print(X_processed_hdd_df)

# Save the processed HDD DataFrame to a CSV file
output_file_path_hdd = 'processed_hdd_data.csv'  # Specify your file path here
X_processed_hdd_df.to_csv(output_file_path_hdd, index=False)


In [ ]:
# Convert X_processed_hdd to a numpy array if it isn't already one
X_dense_hdd = X_processed_hdd.toarray() if hasattr(X_processed_hdd, "toarray") else np.array(X_processed_hdd)

# Replace NaN values with the mean of each column, handling columns with all NaNs separately for HDD data
for i in range(X_dense_hdd.shape[1]):
    column = X_dense_hdd[:, i]
    nan_mask = np.isnan(column)
    if nan_mask.all():
        X_dense_hdd[:, i] = 0  # Replace all NaN columns with 0 or another appropriate value
    else:
        column_mean = np.nanmean(column)
        column[nan_mask] = column_mean  # Replace NaNs in the column with the mean of the non-NaN values

# Ensure no infinite values exist, replacing them if they do for HDD data
inf_mask = np.isinf(X_dense_hdd)
X_dense_hdd[inf_mask] = np.nan  # Convert infinities to NaN
np.nan_to_num(X_dense_hdd, copy=False, nan=0.0)  # Then replace NaNs with 0 (or another value)

# Now proceed with hierarchical clustering for HDD data
linked_hdd = linkage(X_dense_hdd, 'ward')

# Assuming you want to dynamically generate labels for a plot or analysis
# Here we prepare labelList_hdd for potentially labeling points in a plot
labelList_hdd = [f"Point {i+1}" for i in range(X_dense_hdd.shape[0])]


In [ ]:
# Assuming df_hdd is your DataFrame containing the HDD data
# Define numeric and categorical features for HDD
numeric_features_hdd = ['Benchmark']  # Benchmark is a numeric feature for HDD as well
categorical_features_hdd = ['Storage_Size', 'Model_Name']  # Specific features for HDD

# Create pipelines for both numeric and categorical preprocessing for HDD
numeric_transformer_hdd = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer_hdd = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer with appropriate transformations for HDD
preprocessor_hdd = ColumnTransformer(transformers=[
    ('num', numeric_transformer_hdd, numeric_features_hdd),
    ('cat', categorical_transformer_hdd, categorical_features_hdd)])

# Apply preprocessing to the HDD data
X_processed_hdd = preprocessor_hdd.fit_transform(df_hdd)  # This gets you the transformed HDD data

# Convert to dense format if necessary, especially for hierarchical clustering
X_processed_dense_hdd = X_processed_hdd.toarray()

# Hierarchical Clustering to create a linkage matrix 'Z' for HDD data
Z_hdd = linkage(X_processed_dense_hdd, method='ward')

# Plot the dendrogram for visual inspection for HDD data
#plt.figure(figsize=(10, 7))
#dendrogram(Z_hdd)
#plt.title('Hierarchical Clustering Dendrogram for HDD Data')
#plt.xlabel('Sample index')
#plt.ylabel('Distance')
#plt.show()

# Suggesting an optimal distance and finding the number of clusters for HDD data
distances_hdd = Z_hdd[:, 2]
distance_diffs_hdd = np.diff(-np.sort(distances_hdd))
idx_of_largest_gap_hdd = np.argmax(distance_diffs_hdd)
optimal_distance_hdd = distances_hdd[idx_of_largest_gap_hdd]
clusters_hdd = fcluster(Z_hdd, optimal_distance_hdd, criterion='distance')

# Determine the optimal number of clusters using silhouette scores for HDD data
max_silhouette_score_hdd = -1
optimal_num_clusters_hdd = 2
#for num_clusters_hdd in range(2, len(np.unique(clusters_hdd)) + 1):
 #   cluster_labels_hdd = fcluster(Z_hdd, num_clusters_hdd, criterion='maxclust')
  #  silhouette_avg_hdd = silhouette_score(X_processed_dense_hdd, cluster_labels_hdd)
   # if silhouette_avg_hdd > max_silhouette_score_hdd:
    #    max_silhouette_score_hdd = silhouette_avg_hdd
     #   optimal_num_clusters_hdd = num_clusters_hdd

print(f"The optimal number of clusters based on silhouette score for HDD data is: {optimal_num_clusters_hdd}")


In [ ]:
# Apply Agglomerative Clustering to HDD data
cluster_hdd = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')
cluster_labels_hdd = cluster_hdd.fit_predict(X_processed_dense_hdd)  # Use the processed dense HDD data here

print(cluster_labels_hdd)

In [ ]:
# Assuming df_hdd is the original DataFrame and cluster_labels_hdd contains the cluster assignments
df_hdd['cluster_hdd'] = cluster_labels_hdd

# Calculating centroids for HDD data
#HDD_centroids = df_hdd.groupby('cluster_hdd').mean()

#print(HDD_centroids)

REQUEST AND RECIEVE INFORMATION



In [ ]:
"pull benchmarkscore from website"
def get_benchmark_score(component_model, dataset):
    """
    Retrieves the benchmark score for a given component model from the dataset.

    Args:
    - component_model (str): The model identifier of the component.
    - dataset (pd.DataFrame): The dataset containing component data.

    Returns:
    - float: The benchmark score of the component.
    """
    component_data = dataset[dataset['Model'] == component_model]
    if not component_data.empty:
        return component_data.iloc[0]['Benchmark']
    else:
        print(f"Component {component_model} not found in dataset.")
        return None

In [ ]:
"find the match based on model name of the component "
def find_closest_matches(target_scores, dataset):
    """
    Finds the closest matches in the dataset for the target benchmark scores.

    Args:
    - target_scores (list of floats): Target benchmark scores for recommendations.
    - dataset (pd.DataFrame): The dataset containing component data.

    Returns:
    - dict: A dictionary mapping target increases to the closest matching components.
    """
    recommendations = {}
    for target in target_scores:
        # Find components with benchmark scores closest to but not below the target
        matches = dataset[dataset['Benchmark'] >= target].nsmallest(1, 'Benchmark')
        if not matches.empty:
            model = matches.iloc[0]['Model']
            score = matches.iloc[0]['Benchmark']
            recommendations[target] = (model, score)
        else:
            recommendations[target] = ("No match found", 0)
    return recommendations

CPU Recommendation method


In [ ]:
def recommend_CPU_component(start_component_name, increase_percents=[15, 30, 45]):
    global df_cpu  # Assuming df_cpu is globally accessible

    print(f"Received model name from another method: '{start_component_name}'")
    start_component_matches = df_cpu[df_cpu['Model'] == start_component_name]
    if start_component_matches.empty:
        print(f"Model '{start_component_name}' not found in DataFrame.")
        return []

    start_component = start_component_matches.iloc[0]
    start_benchmark = float(start_component['Benchmark'])
    start_cluster = start_component['cluster']
    print(f"Found starting component '{start_component_name}' with Benchmark {start_benchmark} in cluster {start_cluster}.")

    target_benchmarks = [start_benchmark * (1 + p / 100) for p in increase_percents]
    recommendations = []
    cluster_components = df_cpu[df_cpu['cluster'] == start_cluster]

    print(f"Number of components in the same cluster: {len(cluster_components)}")

    for increase_percent, target in zip(increase_percents, target_benchmarks):
        eligible = cluster_components[cluster_components['Benchmark'] >= target]
        print(f"Looking for components with at least {increase_percent}% increase (target benchmark: {target}). Eligible components: {len(eligible)}")

        if not eligible.empty:
            closest = eligible.iloc[(eligible['Benchmark'] - target).abs().argsort()[:1]]
            recommendation = closest[['Model', 'Benchmark']].to_dict('records')[0]
            recommendations.append({"Increase": increase_percent, "Details": recommendation})
        else:
            max_benchmark_component = df_cpu.loc[df_cpu['Benchmark'].idxmax()]
            print(f"No components found for a {increase_percent}% increase. The highest benchmark score in the dataset is by {max_benchmark_component['Model']} with a Benchmark of {max_benchmark_component['Benchmark']}.")
            recommendations.append({"Increase": increase_percent, "Details": {'Model': max_benchmark_component['Model'], 'Benchmark': max_benchmark_component['Benchmark']}})

    # For debugging and verification, display the recommendations more clearly
    for rec in recommendations:
        print(f"Recommendation for {rec['Increase']}% increase: Model '{rec['Details']['Model']}' with Benchmark {rec['Details']['Benchmark']}")

    return recommendations



GPU recommendation method

In [ ]:
def recommend_GPU_component(start_component_name, increase_percents=[15, 30, 45]):
    global df_gpu  # Assuming df_gpu is globally accessible

    # Check if the starting GPU model is in the DataFrame
    start_component_matches = df_gpu[df_gpu['Model'] == start_component_name]
    if start_component_matches.empty:
        print(f"GPU Model '{start_component_name}' not found in DataFrame.")
        return []

    # Extract details of the starting component
    start_component = start_component_matches.iloc[0]
    start_benchmark = float(start_component['Benchmark'])
    if 'cluster' in start_component:
        start_cluster = start_component['cluster']
        print(f"Found starting GPU '{start_component_name}' with Benchmark {start_benchmark} in cluster {start_cluster}.")
    else:
        start_cluster = None
        print(f"Cluster information is not available for '{start_component_name}'.")

    target_benchmarks = [start_benchmark * (1 + p / 100) for p in increase_percents]
    recommendations = []

    # Filter by cluster if available, else consider the whole dataset
    cluster_components = df_gpu[df_gpu['cluster'] == start_cluster] if start_cluster else df_gpu

    for increase_percent, target in zip(increase_percents, target_benchmarks):
        eligible = cluster_components[cluster_components['Benchmark'] >= target]
        print(f"Looking for GPUs with at least {increase_percent}% increase (target benchmark: {target}). Eligible components: {len(eligible)}")

        if not eligible.empty:
            closest = eligible.iloc[(eligible['Benchmark'] - target).abs().argsort()[:1]]
            recommendation = closest[['Model', 'Benchmark']].to_dict('records')[0]
            recommendations.append({"Increase": increase_percent, "Details": recommendation})
        else:
            max_benchmark_component = df_gpu.loc[df_gpu['Benchmark'].idxmax()]
            print(f"No GPUs found for a {increase_percent}% increase. The highest benchmark score in the dataset is by {max_benchmark_component['Model']} with a Benchmark of {max_benchmark_component['Benchmark']}.")
            recommendations.append({"Increase": increase_percent, "Details": {'Model': max_benchmark_component['Model'], 'Benchmark': max_benchmark_component['Benchmark']}})

    # For debugging and verification, clearly display the recommendations
    for rec in recommendations:
        print(f"Recommendation for {rec['Increase']}% increase: Model '{rec['Details']['Model']}' with Benchmark {rec['Details']['Benchmark']}")

    return recommendations


RAM Recommendation method

In [ ]:
def recommend_RAM_component(start_component_name, increase_percents=[15, 30, 45]):
    global df_ram  # Assuming df_ram is globally accessible

    # Check if the starting RAM model exists in the DataFrame
    start_component_matches = df_ram[df_ram['Model'] == start_component_name]
    if start_component_matches.empty:
        print(f"RAM Model '{start_component_name}' not found in DataFrame.")
        return []

    # Extract details of the starting component
    start_component = start_component_matches.iloc[0]
    start_benchmark = float(start_component['Benchmark'])
    start_cluster = start_component.get('cluster_ram', None)
    print(f"Found starting RAM component '{start_component_name}' with Benchmark {start_benchmark} in cluster {start_cluster}.")

    target_benchmarks = [start_benchmark * (1 + p / 100) for p in increase_percents]
    recommendations = []

    cluster_components = df_ram[df_ram['cluster_ram'] == start_cluster] if start_cluster else df_ram

    for increase_percent, target in zip(increase_percents, target_benchmarks):
        eligible_within_cluster = cluster_components[cluster_components['Benchmark'] >= target]

        if not eligible_within_cluster.empty:
            # Identify the closest match within the same cluster
            closest_within_cluster = eligible_within_cluster.iloc[(eligible_within_cluster['Benchmark'] - target).abs().argsort()[:1]]
            recommendation = closest_within_cluster[['Model', 'Benchmark']].to_dict('records')[0]
            recommendations.append({"Increase": increase_percent, "Details": recommendation})
        else:
            # Search the entire dataset if no suitable component within the cluster
            all_eligible = df_ram[df_ram['Benchmark'] >= target]
            if not all_eligible.empty:
                closest_across_all = all_eligible.iloc[(all_eligible['Benchmark'] - target).abs().argsort()[:1]]
                recommendation = closest_across_all[['Model', 'Benchmark']].to_dict('records')[0]
                recommendations.append({"Increase": increase_percent, "Details": recommendation})
            else:
                print(f"No RAM components found for a {increase_percent}% increase.")
                # Consider adding a default case here if needed, such as recommending the highest benchmark RAM in the dataset or within the cluster

    # Print the recommendations with the structured format
    for rec in recommendations:
        print(f"Recommendation for {rec['Increase']}% increase: Model '{rec['Details']['Model']}' with Benchmark {rec['Details']['Benchmark']}")

    return recommendations


SSD recommendation method

In [ ]:
def recommend_SSD_component(start_component_name, increase_percents=[15, 30, 45]):
    global df_ssd  # Access df_ssd globally within the function

    # Check if the starting SSD model exists in the DataFrame
    start_component_matches = df_ssd[df_ssd['Model'] == start_component_name]
    if start_component_matches.empty:
        print(f"SSD Model '{start_component_name}' not found in DataFrame.")
        return []

    # Extract details of the starting component
    start_component = start_component_matches.iloc[0]
    start_benchmark = float(start_component['Benchmark'])  # Ensure start_benchmark is a float
    start_cluster = start_component['cluster_ssd']
    print(f"Found starting SSD component '{start_component_name}' with Benchmark {start_benchmark} in cluster {start_cluster}.")

    # Calculate target benchmarks for the desired performance increases
    target_benchmarks = [start_benchmark * (1 + p / 100) for p in increase_percents]
    recommendations = []

    # Search for eligible components within the same cluster
    cluster_components = df_ssd[df_ssd['cluster_ssd'] == start_cluster]

    for increase_percent, target in zip(increase_percents, target_benchmarks):
        print(f"Looking for SSD components with at least {increase_percent}% increase (target benchmark: {target}).")
        eligible_within_cluster = cluster_components[cluster_components['Benchmark'] >= target]
        print(f"Eligible components within the same cluster: {len(eligible_within_cluster)}")

        if not eligible_within_cluster.empty:
            closest_within_cluster = eligible_within_cluster.iloc[(eligible_within_cluster['Benchmark'] - target).abs().argsort()[:1]]
            recommendation = closest_within_cluster[['Model', 'Benchmark']].to_dict('records')[0]
            recommendations.append({"Increase": increase_percent, "Details": recommendation})
        else:
            all_eligible = df_ssd[df_ssd['Benchmark'] >= target]
            print(f"Eligible components across all clusters: {len(all_eligible)}")

            if not all_eligible.empty:
                closest_across_all = all_eligible.iloc[(all_eligible['Benchmark'] - target).abs().argsort()[:1]]
                recommendation = closest_across_all[['Model', 'Benchmark']].to_dict('records')[0]
                recommendations.append({"Increase": increase_percent, "Details": recommendation})
            else:
                print(f"No SSD components found for a {increase_percent}% increase. The highest benchmark score in the cluster is by {start_component_name} with a Benchmark of {start_benchmark}.")
                recommendation = {'Model': start_component_name, 'Benchmark': start_benchmark}
                recommendations.append({"Increase": increase_percent, "Details": recommendation})

    # Print the recommendations
    for rec in recommendations:
        print(f"Recommendation for {rec['Increase']}% increase: Model '{rec['Details']['Model']}' with Benchmark {rec['Details']['Benchmark']}")

    return recommendations



HDD recommendation method

In [ ]:
def recommend_HDD_component(start_component_name, increase_percents=[15, 30, 45]):
    global df_hdd  # Access df_hdd globally within the function

    # Verify if the starting HDD model is in the DataFrame
    start_component_matches = df_hdd[df_hdd['Model'] == start_component_name]
    if start_component_matches.empty:
        print(f"HDD Model '{start_component_name}' not found in DataFrame.")
        return []

    # Extract details of the starting component
    start_component = start_component_matches.iloc[0]
    start_benchmark = float(start_component['Benchmark'])  # Ensure start_benchmark is a float
    start_cluster = start_component['cluster_hdd']
    print(f"Found starting HDD component '{start_component_name}' with Benchmark {start_benchmark} in cluster {start_cluster}.")

    # Calculate target benchmarks for the desired performance increases
    target_benchmarks = [start_benchmark * (1 + p / 100) for p in increase_percents]
    recommendations = []

    # Search for eligible components within the same cluster
    cluster_components = df_hdd[df_hdd['cluster_hdd'] == start_cluster]

    for increase_percent, target in zip(increase_percents, target_benchmarks):
        print(f"Looking for HDD components with at least {increase_percent}% increase (target benchmark: {target}).")
        eligible_within_cluster = cluster_components[cluster_components['Benchmark'] >= target]
        print(f"Eligible components within the same cluster: {len(eligible_within_cluster)}")

        if not eligible_within_cluster.empty:
            closest_within_cluster = eligible_within_cluster.iloc[(eligible_within_cluster['Benchmark'] - target).abs().argsort()[:1]]
            recommendation = closest_within_cluster[['Model', 'Benchmark']].to_dict('records')[0]
            recommendations.append({"Increase": increase_percent, "Details": recommendation})
        else:
            all_eligible = df_hdd[df_hdd['Benchmark'] >= target]
            print(f"Eligible components across all clusters: {len(all_eligible)}")

            if not all_eligible.empty:
                closest_across_all = all_eligible.iloc[(all_eligible['Benchmark'] - target).abs().argsort()[:1]]
                recommendation = closest_across_all[['Model', 'Benchmark']].to_dict('records')[0]
                recommendations.append({"Increase": increase_percent, "Details": recommendation})
            else:
                print(f"No HDD components found for a {increase_percent}% increase. The highest benchmark score in the cluster is by {start_component_name} with a Benchmark of {start_benchmark}.")
                # Default to the highest benchmark score within the same cluster if no better options are found
                recommendation = {'Model': start_component_name, 'Benchmark': start_benchmark}
                recommendations.append({"Increase": increase_percent, "Details": recommendation})

    # Print the recommendations
    for rec in recommendations:
        print(f"Recommendation for {rec['Increase']}% increase: Model '{rec['Details']['Model']}' with Benchmark {rec['Details']['Benchmark']}")

    return recommendations



In [ ]:
from google.cloud import pubsub_v1
import os
import json
import requests
import logging
import threading

# Initialize Pub/Sub client
publisher = pubsub_v1.PublisherClient()

# Define the topic name where recommendations will be published
output_topic_name = 'projects/watchful-net-416319/topics/PC_Building_Assistant_Receiver'


recommendation_methods = {
    'CPU': recommend_CPU_component,
    'GPU': recommend_GPU_component,
    'RAM': recommend_RAM_component,
    'SSD': recommend_SSD_component,
    'HDD': recommend_HDD_component
}

# Initialize the Pub/Sub client
subscriber = pubsub_v1.SubscriberClient()

# Your Google Cloud project ID
project_id = "watchful-net-416319"

# Existing subscription details
existing_subscription_id = "PC_Building_Assistant"
existing_subscription_path = subscriber.subscription_path(project_id, existing_subscription_id)

# New subscription details for PC_Building_Assistant_Receiver
new_subscription_id = "PC_Building_Assistant_Receiver"
new_subscription_path = subscriber.subscription_path(project_id, new_subscription_id)

function_url ='https://us-central1-watchful-net-416319.cloudfunctions.net/PC_Building_Assistant_Receiver'

def callback_existing(message):
    try:
        print(f"Received message on {existing_subscription_id}: {message}")
        process_message(message)
    except KeyError as e:
        print(f"KeyError occurred: {str(e)} - Possible missing data in message.")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
    finally:
        message.ack()  # Acknowledge the message to prevent further retries


def callback_new(message):
    """Callback function for the new subscription."""
    print(f"Received message on {new_subscription_id}: {message}")
    # Process the message similarly as before
    process_message(message)

def process_message(message):
    """Process message common logic."""
    data = message.data.decode("utf-8")
    payload = json.loads(data)
    model = payload.get("model")
    componentType = payload.get("componentType", payload.get("type"))  # Fallback to "type" if "componentType" is missing
    email = payload.get("email")  # Assuming email is correctly provided in the message
    username = payload.get("username")  # Correctly retrieve username from the message

    if not componentType:
        print(f"Error: Component type is missing or None for model {model}. Message payload: {payload}")
        return  # Exit the function if component_type is missing or None

    if componentType in recommendation_methods:
        print(f"Processing recommendation for Model: {model}, Component Type: {componentType}")
        print(f"User: {username}, Email: {email}")  # Log username and email

        # Assume recommendation_methods[component_type](model) is defined elsewhere
        recommendation_result = recommendation_methods[componentType](model)

        recommendation_message = {
            "model": model,
            "componentType": componentType,
            "recommendation": recommendation_result,
            "email": email,
            "username": username  # Correctly include username in the output message
        }

        publish_recommendation(recommendation_message)
    else:
        print(f"Unknown component type: {componentType}")

def publish_recommendation(recommendation):
    """Sends the recommendation to the Cloud Function via HTTP POST."""
    try:
        headers = {'Content-Type': 'application/json'}
        print("Attempting to send recommendation...")

        # Send the POST request to the Cloud Function endpoint
        response = requests.post(function_url, data=json.dumps(recommendation), headers=headers)

        if response.status_code == 200:
            print("Recommendation sent to Cloud Function successfully.")
        else:
            print(f"Failed to send recommendation, status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred while sending the recommendation: {e}")


# Subscribe to the existing subscription
subscriber_future_existing = subscriber.subscribe(existing_subscription_path, callback_existing)
threading.Thread(target=lambda: subscriber_future_existing.result()).start()

# Subscribe to the new subscription
subscriber_future_new = subscriber.subscribe(new_subscription_path, callback_new)
threading.Thread(target=lambda: subscriber_future_new.result()).start()

print(f"Listening for messages on {existing_subscription_id} and {new_subscription_id}...\n")

try:
    # Keep the main thread running
    while True:
        pass
except KeyboardInterrupt:
    print("Interrupted, stopping...")
    subscriber_future_existing.cancel()
    subscriber_future_new.cancel()
import time
while True:
    time.sleep(60)
